In [1]:
import sys

import spacy
import medspacy
import nltk

from medspacy.util import DEFAULT_PIPENAMES
from medspacy.visualization import visualize_ent
from medspacy.section_detection import Sectionizer
medspacy_pipes = DEFAULT_PIPENAMES.copy()

if 'medspacy_quickumls' not in medspacy_pipes:
    medspacy_pipes.add('medspacy_quickumls')
   
print(medspacy_pipes)
   
nlp = medspacy.load(enable = medspacy_pipes, quickumls_path='/home/zhichaoyang/medspacy_test/')
print(nlp.pipe_names)

{'medspacy_quickumls', 'medspacy_target_matcher', 'medspacy_pyrush', 'medspacy_tokenizer', 'medspacy_context'}


/home/wjang/anaconda3/envs/transformer_limitation/lib/python3.9/site-packages/quickumls_simstring/simstring.py:15: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


['medspacy_pyrush', 'medspacy_target_matcher', 'medspacy_quickumls', 'medspacy_context']


In [2]:
from medspacy.util import DEFAULT_PIPENAMES
import medspacy
import numpy as np
import pandas as pd
from pathlib import Path
import os, sys
import jsonlines

project_path = Path().absolute().parents[0].joinpath("src")
os.sys.path.append(project_path.as_posix())

In [3]:
from utils import load_config
# from quickumls.spacy_component import SpacyQuickUMLS

config = load_config()
projectPath = Path(config['project_path'])
dataPath = projectPath.joinpath("data")
outputPath = dataPath.joinpath('processed/')
contextPath = dataPath.joinpath('processed/')
semantic_path = dataPath.joinpath("raw/medspacy_test/SemanticTypes_2018AB.txt")
quickumls_path = "/home/zhichaoyang/medspacy_test/"


def load_semantic_types(path) :
    header = ["abrev", "TUI", "text"]
    df = pd.read_csv(path,sep="|",names = header)
    return df


def load_contexts(name) :
    '''
    names : dev, train, test
    '''
    path = contextPath.joinpath(name +".jsonl")

    with jsonlines.open(path) as f:
        contexts = []
        for line in f.iter():
            contexts.append(line['context'])# or whatever else you'd like to do
    return contexts


def load_spacy() :

    medspacy_pipes = DEFAULT_PIPENAMES.copy()
    if 'medspacy_quickumls' not in medspacy_pipes:
        medspacy_pipes.add('medspacy_quickumls')
    nlp = medspacy.load(enable = medspacy_pipes, quickumls_path=quickumls_path)
    return nlp


def extract_medical_concepts(extractor, text):
    context = extractor(text)
    the_enteties = [all_ent for all_ent in context.ents]
    return the_enteties


In [4]:
nlp = load_spacy()
context = load_contexts("dev")
sample = context[12]

doc = nlp(sample)

/home/wjang/anaconda3/envs/transformer_limitation/lib/python3.9/site-packages/medspacy/common/medspacy_matcher.py:80: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = self.matcher(doc)


In [5]:
text = ".\n".join(sample.split(". "))
doc = nlp(text)

/home/wjang/anaconda3/envs/transformer_limitation/lib/python3.9/site-packages/medspacy/common/medspacy_matcher.py:80: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = self.matcher(doc)


In [6]:
visualize_ent(doc)

/home/wjang/anaconda3/envs/transformer_limitation/lib/python3.9/site-packages/spacy/displacy/__init__.py:64: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [7]:
entities = set()
for ent in doc :
    ent_type = ent.ent_type_
    if ent_type != '' :
        entities.add(ent_type)
        # print("added entity ",ent.ent_type_)

# make combinations
from itertools import combinations, product

combobj = product(entities,repeat=2)
# for idx, comb in enumerate(combobj) :
#     print(idx, comb)


Now, let's create a graph based on the extracted entities

In [8]:
import neo4j
from neo4j import GraphDatabase, RoutingControl


URI = "neo4j://localhost:7687"
AUTH = ("neo4j", "q1q2q3q4")

driver = GraphDatabase.driver(URI, auth=AUTH)
driver.verify_connectivity()

In [9]:
def query_neo4j(n1, n2):
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        answer = driver.execute_query(
            f"""
            match (n1)-[r]->(n2) where n1.CUI='{n1}' and n2.CUI='{n2}' return n1.CUI, n1.name, type(r), n2.CUI, n2.name
            """,
            result_transformer_=neo4j.Result.to_df
        )
    return answer

In [10]:
relations = []
for comb in combobj :

    # continue if the two concepts are the same..
    if comb[0] == comb[1] :
        continue
    answer = query_neo4j(comb[0], comb[1])
    relations.append(answer)
    # break

# relations = list(filter(lambda x : x.records != [], relations))

In [93]:
df = pd.concat(relations, ignore_index=True)

ValueError: No objects to concatenate

In [85]:
df.iloc[11]

n1.CUI                 C0031809
n1.name    Physical Examination
r                            ()
n2.CUI                 C0031809
n2.name    Physical Examination
Name: 11, dtype: object

In [84]:
df.iloc[11]['r']

<Relationship element_id='2408682' nodes=(<Node element_id='377832' labels=frozenset() properties={}>, <Node element_id='377832' labels=frozenset() properties={}>) type='classifies' properties={}>

In [62]:
from neo4j import graph

type(relations[0])

neo4j._work.eager_result.EagerResult

In [87]:
import pyvis

def visualize_result(query_graph, nodes_text_properties):
    visual_graph = pyvis.network.Network()

    for node in query_graph.nodes:
        node_label = list(node.labels)[0]
        node_text = node[nodes_text_properties[node_label]]
        visual_graph.add_node(node.element_id, node_text, group=node_label)

    for relationship in query_graph.relationships:
        visual_graph.add_edge(
            relationship.start_node.element_id,
            relationship.end_node.element_id,
            title=relationship.type
        )

    visual_graph.show('network.html', notebook=False)

In [83]:
relations[0][0][0]

neo4j._data.Record

In [30]:
example = relations[0]

example

IndexError: list index out of range

[]

In [36]:
import neo4jupyter
neo4jupyter.init_notebook_mode()

<IPython.core.display.Javascript object>

In [37]:
neo4jupyter.draw(relations[0])

TypeError: draw() missing 1 required positional argument: 'options'

In [ ]:
def entity_overlap_scores(predictions, references):
    # init 
    tret_sem_ids = ['T059', 'T060', 'T061', 'T058', 'T056'] #test and treatment
    symp_sem_ids = ['T184', 'T034', 'T037', 'T033'] # symptom
    dise_sem_ids = ['T020', 'T019','T046', 'T047', 'T048', 'T191', 'T049', 'T050'] #disease
    drug_sem_ids = ['T073', 'T074', 'T203', 'T075', 'T200', 'T192'] #drug
    sust_sem_ids = ['T120', 'T121', 'T195', 'T122', 'T123', 'T125', 'T126', 'T127', 'T129', 'T130', 'T131', 'T104', 'T109', 'T114', 'T116', 'T197', 'T196', 'T168'] # substance
    cuitypes_toinclude = tret_sem_ids + symp_sem_ids + dise_sem_ids + drug_sem_ids + sust_sem_ids
    # df = pd.read_csv('/home/zhichaoyang/medspacy_test/SemanticTypes_2018AB.txt', sep="|", names=['abrev','TUI','text'])
    # tui2abrev = dict(zip(df.TUI, df.abrev))
    medspacy_pipes = DEFAULT_PIPENAMES.copy()
    if 'medspacy_quickumls' not in medspacy_pipes: 
        medspacy_pipes.add('medspacy_quickumls')
    print(medspacy_pipes)
    nlp = medspacy.load(enable = medspacy_pipes, quickumls_path='/home/zhichaoyang/medspacy_test/')

    results = {"concept_recall": [], "concept_precision": [], "concept_f1": []}
    assert len(predictions) == len(references)
    # compare
    for pred, reff in zip(predictions, references):
        pred_cuis = []
        doc = nlp(pred)
        for ent in doc.ents:
            for a in ent._.semtypes:
                if a in cuitypes_toinclude:
                    pred_cuis.append(ent.label_)
                    break
            # print('Entity text : {}'.format(ent.text))
            # print('Label (UMLS CUI) : {}'.format(ent.label_))
            # print('Similarity : {}'.format(ent._.similarity))
            # print('Semtypes : {}'.format(ent._.semtypes))
        reff_cuis = []
        doc = nlp(reff)
        for ent in doc.ents:
            for a in ent._.semtypes:
                if a in cuitypes_toinclude:
                    reff_cuis.append(ent.label_)
                    break
        num_intersect = len(set(pred_cuis).intersection(set(reff_cuis)))
        num_rec = len(set(reff_cuis))
        num_pre = len(set(pred_cuis))
        rec = float(num_intersect)/(num_rec + 0.000001) 
        pre = float(num_intersect)/(num_pre + 0.000001) 
        f1 = (2 * pre * rec) / (pre + rec + 0.000001) 
        results["concept_recall"].append(rec)
        results["concept_precision"].append(pre)
        results["concept_f1"].append(f1)
    results = {"concept_recall": np.array(results["concept_recall"]).mean()*100, 
        "concept_precision": np.array(results["concept_precision"]).mean()*100, 
        "concept_f1": np.array(results["concept_f1"]).mean()*100,
        "concept_f1s": np.array(results["concept_f1"])
        return 0